In [624]:
from dataclasses import dataclass
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
import astropy.units as u
from astropy.constants import G
from functools import lru_cache
import numpy as np
from bs4 import BeautifulSoup
import requests
from types import MethodType
import pandas as pd
from copy import deepcopy

@lru_cache
def get_nexsci_tab(name):
    """LRU Cached helper to get the composite entry for a given name"""
    return NasaExoplanetArchive.query_object(name, table='pscomppars')

@lru_cache
def get_citation(bibcode):
    """Goes to NASA ADS and webscrapes the bibtex citation for a give bibcode"""
    d = requests.get(f'https://ui.adsabs.harvard.edu/abs/{bibcode}/exportcitation')
    soup = BeautifulSoup(d.content, 'html.parser')
    return soup.find('textarea').text

def get_ref_dict(tab):
    """Parses the NExSci table for a list of references"""
    cols = [c for c in tab.columns if 'reflink' in c]
    refs = np.unique(tab[cols])[0]
    result = {ref.split('>')[1].split('</a')[0].strip(): ref.split('href=')[1].split(' target=ref')[0] for ref in refs if ref != ''}
    for key, item in result.items():
        if 'ui.adsabs' in item.lower():
            result[key] = get_citation(item.split('abs/')[1].split('/')[0])
    return result

In [519]:
class Planet(object):
    """Helper class to get information from NExSci. This class only holds and prints information, it doesn't calculate anything."""
    
    def __init__(self, hostname:str, letter:str='b'):
        self.hostname = hostname
        self.letter = letter
        tab = get_nexsci_tab(hostname)
        tab = tab[tab['pl_letter'] == letter]
        if len(tab) == 0:
            raise ValueError('No planet found')
        self._tab = tab[0]
        _ = [setattr(self, c, self._tab[c].filled(np.nan) if isinstance(self._tab[c], u.Quantity) else u.Quantity(self._tab[c]))
                 if isinstance(self._tab[c], (u.Quantity, float, int)) else setattr(self, c, self._tab[c])
             for c in list(self._tab.columns) if not (c.endswith('err1') | c.endswith('err2') | c.endswith('reflink') | c.endswith('lim') | c.endswith('str'))]
        # Error on the archive that the unit is days not hours...!
        if (self.pl_trandur.unit == u.day):
            self.pl_trandur /= 24    

        for c in self._tab.columns:
            if c.endswith('reflink'):
                attr = getattr(self, '_'.join(c.split('_')[:-1]))
                if isinstance(attr, u.Quantity):
                    if self._tab[c] != '':
                        ref = self._tab[c].split('href=')[1].split(' target=ref')[0]
                        if 'ui.adsabs' in ref.lower():
                            ref = get_citation(ref.split('abs/')[1].split('/')[0])
                            setattr(attr, 'reference', ref)
                            setattr(attr, 'reference_name', ref.split('{')[1].split(',')[0])
                            setattr(attr, 'reference_link', ref.split('adsurl = {')[1].split('}')[0])
            if c.endswith('err1'):
                attr = getattr(self, c[:-4])
                if isinstance(attr, u.Quantity):
                    if self._tab[c] != '':
                        setattr(attr, 'err1', u.Quantity(self._tab[c[:-4] + 'err1']))
                        setattr(attr, 'err2', u.Quantity(self._tab[c[:-4] + 'err2']))
                        setattr(attr, 'err', u.Quantity([self._tab[c[:-4] + 'err1'], -self._tab[c[:-4] + 'err2']]).mean())
                    else:
                        setattr(self._tab[c], 'err', np.nan*self._tab[c].unit)
            if c.endswith('lim'):
                # Any "limit" parameters need to be set to nans
                if self._tab[c] == 1:
                    attr = getattr(self, c[:-3])
                    attr *= np.nan
                    for e in ['err1', 'err2', 'err']:
                        if hasattr(attr, e):
                            setattr(attr, e, getattr(attr, e) * np.nan)
        self._fix_eorj()
        self._fix_orbsmax()
        self._fix_ratdor()
        self._fix_ratror()
        self._fix_eqt()
        self.references = get_ref_dict(self._tab)
        self.acknowledgements = ['This research has made use of the NASA Exoplanet Archive, which is operated by the California Institute of Technology, under contract with the National Aeronautics and Space Administration under the Exoplanet Exploration Program.']

    @property
    def name(self):
        return self.hostname + self.letter
    
    def _fix_eorj(self):
        if np.isfinite(self.pl_bmasse) ^ np.isfinite(self.pl_bmassj):
            if np.isfinite(self.pl_bmasse):
                self.pl_bmassj = self.pl_bmasse.to(u.jupiterMass)
                for e in ['err1', 'err2', 'err']:
                    if hasattr(self.pl_bmasse, e):
                        setattr(self.pl_bmassj, e, getattr(self.pl_bmasse, e).to(u.jupiterMass))
                self.pl_bmassj.reference = self.pl_bmasse.reference
            else:
                self.pl_bmasse = self.pl_bmassj.to(u.earthMass)
                for e in ['err1', 'err2', 'err']:
                    if hasattr(self.pl_bmassj, e):
                        setattr(self.pl_bmasse, e, getattr(self.pl_bmassj, e).to(u.earthMass))
                self.pl_bmasse.reference = self.pl_bmassj.reference
                
        if np.isfinite(self.pl_rade) ^ np.isfinite(self.pl_radj):
            if np.isfinite(self.pl_rade):
                self.pl_radj = self.pl_rade.to(u.jupiterRad)
                for e in ['err1', 'err2', 'err']:
                    if hasattr(self.pl_rade, e):
                        setattr(self.pl_radj, e, getattr(self.pl_rade, e).to(u.jupiterRad))        
            else:
                self.pl_rade = self.pl_radj.to(u.earthRad)
                for e in ['err1', 'err2', 'err']:
                    if hasattr(self.pl_radj, e):
                        setattr(self.pl_rade, e, getattr(self.pl_radj, e).to(u.earthRad))
                
    
    def _fix_orbsmax(self):
        if not np.isfinite(self.pl_orbsmax):
            a = u.Quantity((((G * self.st_mass)/(4*np.pi) * self.pl_orbper**2)**(1/3)).to(u.AU))
            a.err = ((self.st_mass.err/self.st_mass)**2 + (self.pl_orbper.err/self.pl_orbper)**2)**0.5 * a
            a.reference = 'Calculated'
            self.pl_orbsmax = a
            
    def _fix_ratdor(self):
        if not np.isfinite(self.pl_ratdor):
            q = u.Quantity(self.pl_orbsmax.to(u.AU)/self.st_rad.to(u.AU))
            q.err = (((self.pl_orbsmax.err/self.pl_orbsmax)**2 + (self.st_rad.err/self.st_rad)**2))**0.5 * q
            q.reference = 'Calculated'
            self.pl_ratdor = q

    def _fix_eqt(self):
        if not np.isfinite(self.pl_eqt):
            # Assume albedo is 1
            eqt = self.st_teff * np.sqrt(0.5 * 1/self.pl_ratdor)
            eqt.err = (self.st_teff + self.st_teff.err) * np.sqrt(0.5 * 1/(self.pl_ratdor - self.pl_ratdor.err)) - eqt
            eqt.reference = 'Calculated'
            self.pl_eqt = eqt

    def _fix_ratror(self):
        if not np.isfinite(self.pl_ratror):
            r = self.pl_rade.to(u.solRad)/self.st_rad.to(u.solRad)
            r.err = ((self.pl_rade.err/self.pl_rade)**2 + (self.st_rad.err/self.st_rad)**2)**0.5 * r
            self.pl_ratror = r
        
    def __repr__(self):
         return self.hostname + self.letter

    @property
    def StarParametersTable(self):
        d = pd.DataFrame(columns=['Value', 'Description', 'Reference'])
        for key, symbol, desc in zip(['st_rad', 'st_mass', 'st_age', 'st_logg'],
                                     ['R', 'M', 'Age', 'logg'],
                                     ['Stellar Radius', 'Stellar Mass', "Stellar Age", "Stellar Gravity"]):
            attr = getattr(self, key)
            if np.isfinite(attr):
                d.loc[symbol, 'Value'] = "{0}^{{{1}}}_{{{2}}}".format(attr.to_string(format='latex'), attr.err1.value, attr.err2.value)
                d.loc[symbol, 'Description'] = desc
                d.loc[symbol, 'Reference'] = f"\\cite{{{attr.reference_name}}}"
        return d

    @property
    def StarParametersTableLatex(self):
        print(self.StarParametersTable.to_latex(caption=f'Stellar Parameters for {self.hostname}', label='tab:stellarparams'))


    @property
    def PlanetParametersTable(self):
        d = pd.DataFrame(columns=['Value', 'Description', 'Reference'])
        for key, symbol, desc in zip(['pl_radj', 'pl_bmassj', 'pl_orbper', 'pl_tranmid'],
                                     ['R', 'M', 'P', 'T_0'],
                                     ['Planet Radius', 'Planet Mass', "Planet Orbital Period", "Planet Transit Midpoint"]):
            attr = getattr(self, key)
            if np.isfinite(attr):
                d.loc[symbol, 'Value'] = "{0}^{{{1}}}_{{{2}}}{3}".format(attr.value, attr.err1.value, attr.err2.value, attr.unit.to_string('latex'))
                d.loc[symbol, 'Description'] = desc
                d.loc[symbol, 'Reference'] = f"\\cite{{{attr.reference_name}}}" if hasattr(attr, 'reference_name') else ''
        return d

    @property
    def PlanetParametersTableLatex(self):
        print(self.PlanetParametersTable.to_latex(caption=f'Planet Parameters for {self.hostname + self.letter}', label='tab:planetparams'))


In [629]:
class Planets(object):
    """Special class to hold many planets in one system"""
    def __init__(self, hostname:str):
        self.hostname = hostname
        tab = get_nexsci_tab(hostname)
        if len(tab) == 0:
            raise ValueError('No planets found')
        self.letters = np.unique(list(tab['pl_letter']))
        self.planets = [Planet(self.hostname, letter) for letter in self.letters]
        self._tab = tab
        self._cols = [c for c in list(self._tab.columns) if not (c.endswith('err1') | c.endswith('err2') | c.endswith('reflink') | c.endswith('lim') | c.endswith('str'))]
        _ = [setattr(self, attr, [getattr(planet, attr) for planet in self]) for attr in self._cols if attr.startswith('pl')]
        _ = [setattr(self, attr, getattr(self[0], attr)) for attr in self._cols if attr.startswith('st')]
        _ = [setattr(self, attr, getattr(self[0], attr)) for attr in self._cols if attr.startswith('sy')]
        self.acknowledgements = []
        self.references = self[0].references
        for planet in self:
            self.references.update(planet.references)
            [self.acknowledgements.append(a) for a in planet.acknowledgements]
        self.acknowledgements = list(np.unique(self.acknowledgements))

        
    def __len__(self):
        return len(self.letters)
    
    def __repr__(self):
        return f"{self.hostname} System ({len(self)} Planet{'s' if len(self) > 1 else ''})"

    def __getitem__(self, idx):
        if isinstance(idx, int):
            return self.planets[idx]
        elif isinstance(idx, str):
            if idx in self.letters:
                return self.planets[np.where(self.letters == idx)[0][0]]
            else:
                raise ValueError(f"No planet `{idx}` in the {self.hostname} system.")
        else:
            raise ValueError(f"Can not parse `{idx}` as a planet.")

    @property
    def StarParametersTable(self):
        return self[0].StarParametersTable

    @property
    def StarParametersTableLatex(self):
        return self[0].StarParametersTableLatex

    @property
    def PlanetsParametersTable(self):
        dfs = [planet.PlanetParametersTable[['Value', 'Reference']].rename({'Value':planet.name}, axis='columns') for planet in self]
        return pd.concat([self[0].PlanetParametersTable[['Description']], *dfs], axis=1)

    @property
    def PlanetsParametersTableLatex(self):
        print(self.PlanetsParametersTable.to_latex(caption=f'Planet Parameters for {self.hostname} Planets', label='tab:planetparams'))



In [633]:
self = Planets('HAT-P-11')

In [634]:
self.acknowledgements

['This research has made use of the NASA Exoplanet Archive, which is operated by the California Institute of Technology, under contract with the National Aeronautics and Space Administration under the Exoplanet Exploration Program.']

In [635]:
self.references

{'TICv8': '@ARTICLE{2019AJ....158..138S,\n       author = {{Stassun}, Keivan G. and {Oelkers}, Ryan J. and {Paegert}, Martin and {Torres}, Guillermo and {Pepper}, Joshua and {De Lee}, Nathan and {Collins}, Kevin and {Latham}, David W. and {Muirhead}, Philip S. and {Chittidi}, Jay and {Rojas-Ayala}, B{\\\'a}rbara and {Fleming}, Scott W. and {Rose}, Mark E. and {Tenenbaum}, Peter and {Ting}, Eric B. and {Kane}, Stephen R. and {Barclay}, Thomas and {Bean}, Jacob L. and {Brassuer}, C.~E. and {Charbonneau}, David and {Ge}, Jian and {Lissauer}, Jack J. and {Mann}, Andrew W. and {McLean}, Brian and {Mullally}, Susan and {Narita}, Norio and {Plavchan}, Peter and {Ricker}, George R. and {Sasselov}, Dimitar and {Seager}, S. and {Sharma}, Sanjib and {Shiao}, Bernie and {Sozzetti}, Alessandro and {Stello}, Dennis and {Vanderspek}, Roland and {Wallace}, Geoff and {Winn}, Joshua N.},\n        title = "{The Revised TESS Input Catalog and Candidate Target List}",\n      journal = {\\aj},\n     keyword

In [610]:
self.StarParametersTable

,Value,Description,Reference
R,$0.68 \; \mathrm{Rsun}$^{0.01}_{-0.01},Stellar Radius,\cite{2018AJ....155..255Y}
M,$0.81 \; \mathrm{Msun}$^{0.02}_{-0.03},Stellar Mass,\cite{2018AJ....155..255Y}
Age,$6.5 \; \mathrm{Gyr}$^{5.9}_{-4.1},Stellar Age,\cite{2018AJ....155..255Y}
logg,$4.66 \; \mathrm{}$^{0.01}_{-0.01},Stellar Gravity,\cite{2011MNRAS.417.2166S}


In [593]:
self[0]

HAT-P-11b

['pl_name',
 'pl_letter',
 'hostname',
 'hd_name',
 'hip_name',
 'tic_id',
 'disc_pubdate',
 'disc_year',
 'discoverymethod',
 'disc_locale',
 'disc_facility',
 'disc_instrument',
 'disc_telescope',
 'disc_refname',
 'ra',
 'dec',
 'glon',
 'glat',
 'elon',
 'elat',
 'pl_orbper',
 'pl_orblper',
 'pl_orbsmax',
 'pl_orbincl',
 'pl_orbtper',
 'pl_orbeccen',
 'pl_eqt',
 'pl_occdep',
 'pl_insol',
 'pl_dens',
 'pl_trandep',
 'pl_tranmid',
 'sy_pmdec',
 'sy_plx',
 'sy_dist',
 'sy_bmag',
 'sy_vmag',
 'sy_jmag',
 'sy_hmag',
 'sy_kmag',
 'sy_umag',
 'sy_rmag',
 'sy_imag',
 'sy_zmag',
 'sy_w1mag',
 'sy_w2mag',
 'sy_w3mag',
 'sy_w4mag',
 'sy_gmag',
 'sy_gaiamag',
 'sy_tmag',
 'pl_controv_flag',
 'pl_orbtper_systemref',
 'pl_tranmid_systemref',
 'st_metratio',
 'st_spectype',
 'sy_kepmag',
 'st_rotp',
 'pl_projobliq',
 'gaia_id',
 'cb_flag',
 'pl_trandur',
 'pl_rvamp',
 'pl_radj',
 'pl_rade',
 'pl_ratror',
 'pl_ratdor',
 'pl_trueobliq',
 'sy_icmag',
 'dkin_flag',
 'pl_imppar',
 'pl_bmassj',
 'pl_bm

,Description
R,Planet Radius
M,Planet Mass
P,Planet Orbital Period
T_0,Planet Transit Midpoint


In [440]:
self = Planet('HD209458', 'b')

In [441]:
self.PlanetParametersTable

,Value,Description,Reference
R,1.39^{0.02}_{-0.02}$\mathrm{R_{\rm J}}$,Planet Radius,\cite{2017AJ....153..136S}
M,0.73^{0.04}_{-0.04}$\mathrm{M_{\rm J}}$,Planet Mass,\cite{2017AJ....153..136S}
P,3.52474859^{3.8e-07}_{-3.8e-07}$\mathrm{d}$,Planet Orbital Period,\cite{2017AJ....153..136S}
T_0,2451659.93742^{2e-05}_{-2e-05}$\mathrm{d}$,Planet Transit Midpoint,\cite{2012ApJ...757...18A}


In [443]:
self.PlanetParametersTableLatex

\begin{table}
\caption{Planet Parameters for HD 209458b}
\label{tab:planetparams}
\begin{tabular}{llll}
\toprule
 & Value & Description & Reference \\
\midrule
R & 1.39^{0.02}_{-0.02}$\mathrm{R_{\rm J}}$ & Planet Radius & \cite{2017AJ....153..136S} \\
M & 0.73^{0.04}_{-0.04}$\mathrm{M_{\rm J}}$ & Planet Mass & \cite{2017AJ....153..136S} \\
P & 3.52474859^{3.8e-07}_{-3.8e-07}$\mathrm{d}$ & Planet Orbital Period & \cite{2017AJ....153..136S} \\
T_0 & 2451659.93742^{2e-05}_{-2e-05}$\mathrm{d}$ & Planet Transit Midpoint & \cite{2012ApJ...757...18A} \\
\bottomrule
\end{tabular}
\end{table}



In [413]:
d = pd.DataFrame(columns=['Value', 'Description', 'Reference'])
for key, symbol, desc in zip(['st_rad', 'st_mass', 'st_age', 'st_logg'],
                             ['R', 'M', 'Age', 'logg'],
                             ['Stellar Radius', 'Stellar Mass', "Stellar Age", "Stellar Gravity"]):
    attr = getattr(self, key)
    if np.isfinite(attr):
        d.loc[symbol, 'Value'] = "{0}^{{{1}}}_{{{2}}}".format(attr.to_string(format='latex'), attr.err1.value, attr.err2.value)
        d.loc[symbol, 'Description'] = desc
        d.loc[symbol, 'Reference'] = f"\\cite{{{attr.reference_name}}}"
print(d.to_latex(caption=f'Stellar Parameters for {self.hostname}', label='tab:stellarparams'))

\begin{table}
\caption{Stellar Parameters for HD 209458}
\label{tab:stellarparams}
\begin{tabular}{llll}
\toprule
 & Value & Description & Reference \\
\midrule
R & $1.19 \; \mathrm{Rsun}$^{0.02}_{-0.02} & Stellar Radius & \cite{2017AJ....153..136S} \\
M & $1.23 \; \mathrm{Msun}$^{0.09}_{-0.09} & Stellar Mass & \cite{2017AJ....153..136S} \\
Age & $3.1 \; \mathrm{Gyr}$^{0.8}_{-0.7} & Stellar Age & \cite{2017A&A...602A.107B} \\
logg & $4.45 \; \mathrm{}$^{0.02}_{-0.02} & Stellar Gravity & \cite{2017AJ....153..136S} \\
\bottomrule
\end{tabular}
\end{table}



,Value,Description,Reference
R,$1.19 \; \mathrm{Rsun}$^{0.02}_{-0.02},Stellar Radius,\cite{2017AJ....153..136S}
M,$1.23 \; \mathrm{Msun}$^{0.09}_{-0.09},Stellar Mass,\cite{2017AJ....153..136S}
Age,$3.1 \; \mathrm{Gyr}$^{0.8}_{-0.7},Stellar Age,\cite{2017A&A...602A.107B}
logg,$4.45 \; \mathrm{}$^{0.02}_{-0.02},Stellar Gravity,\cite{2017AJ....153..136S}


In [406]:
print(d.to_latex())

\begin{tabular}{llll}
\toprule
 & Value & Description & Reference \\
\midrule
R & $1.19 \; \mathrm{Rsun}$^{0.02}_{-0.02} & Stellar Radius & \cite{2017AJ....153..136S} \\
M & $1.23 \; \mathrm{Msun}$^{0.09}_{-0.09} & Stellar Mass & \cite{2017AJ....153..136S} \\
Age & $3.1 \; \mathrm{Gyr}$^{0.8}_{-0.7} & Stellar Age & \cite{2017A&A...602A.107B} \\
logg & $4.45 \; \mathrm{}$^{0.02}_{-0.02} & Stellar Gravity & \cite{2017AJ....153..136S} \\
\bottomrule
\end{tabular}



In [380]:
self.st_logg

<Quantity 4.45>

In [381]:
d

,Value,Description,Reference
R,$1.19 \; \mathrm{Rsun}$^{0.02}_{-0.02},Stellar Radius,\cite{2017AJ....153..136S}
M,$1.23 \; \mathrm{Msun}$^{0.09}_{-0.09},Stellar Mass,\cite{2017AJ....153..136S}


ValueError: If using all scalar values, you must pass an index

In [290]:
self.pl_tranmid

<Quantity 2451659.93742 d>

In [265]:
if np.isfinite(self.pl_rade) ^ np.isfinite(self.pl_radj):
    if np.isfinite(self.pl_rade):
        self.pl_radj = self.pl_rade.to(u.jupiterRad)
        for e in ['err1', 'err2', 'err']:
            if hasattr(self.pl_rade, e):
                setattr(self.pl_radj, e, getattr(self.pl_rade, e).to(u.jupiterRad))        
    else:
        self.pl_rade = self.pl_radj.to(u.earthRad)
        for e in ['err1', 'err2', 'err']:
            if hasattr(self.pl_radj, e):
                setattr(self.pl_rade, e, getattr(self.pl_radj, e).to(u.earthRad))
        

AttributeError: 'Quantity' object has no 'reference' member

In [255]:
self.pl_eqt

<Quantity 93.52777979 K>

In [244]:
self.pl_bmassj *= np.nan

In [245]:
self.st_teff

<Quantity 4780. K>

<Quantity 97.29099485 K>

In [216]:
self.pl_bmasse

<Quantity 731.009 earthMass>

In [217]:
if np.isfinite(self.pl_bmasse) ^ np.isfinite(self.pl_bmassj):
    if np.isfinite(self.pl_bmasse):
        self.pl_bmassj = self.pl_bmasse.to(u.jupiterMass)
        for e in ['err1', 'err2', 'err']:
            if hasattr(self.pl_bmasse, e):
                setattr(self.pl_bmassj, e, getattr(self.pl_bmasse, e).to(u.jupiterMass))
        self.pl_bmassj.reference = self.pl_bmasse.reference
    else:
        self.pl_bmasse = self.pl_bmassj.to(u.earthMass)
        for e in ['err1', 'err2', 'err']:
            if hasattr(self.pl_bmassj, e):
                setattr(self.pl_bmasse, e, getattr(self.pl_bmassj, e).to(u.earthMass))
        self.pl_bmasse.reference = self.pl_bmassj.reference
    

<Quantity 2.30001153 jupiterMass>

In [125]:
p.pl_ratdor

<Quantity nan>

In [122]:
q

<Quantity 1306.00412194>

In [86]:
p.pl_orbsmax

<Quantity 4.13 AU>

In [90]:
p.pl_orb

AttributeError: 'Planet' object has no attribute 'pl_orb'

<MaskedQuantity 6.53446814 AU>

<MaskedQuantity 5.74976261 AU>

<Quantity 3132. d>

In [42]:
p.st_mass.err

<Quantity 0.025 Msun>

In [45]:
p.pl_orbper.err

<Quantity 275. d>

In [ ]:
p.pl_eqt

In [65]:
np.round(np.log10(p.pl_orbper.err.value)).astype(int)

/var/folders/bv/0t7fjlgx0mx3_3bdhxqzd7bmh_67k0/T/ipykernel_12147/3898938082.py:1: RuntimeWarning: invalid value encountered in cast
  np.round(np.log10(p.pl_orbper.err.value)).astype(int)


0

In [ ]:
p.pl_bmassj.err

In [451]:
p.references.keys()

dict_keys(['TICv8', 'Hartman et al. 2011', 'ExoFOP-TESS TOI', 'Bonomo et al. 2017'])

In [129]:
p.acknowledgements

['This research has made use of the NASA Exoplanet Archive, which is operated by the California Institute of Technology, under contract with the National Aeronautics and Space Administration under the Exoplanet Exploration Program.']

In [53]:
np.issubdtype(type('christina'), str)

True

True